In [391]:
from datetime import datetime

import numpy as np

def starttimer():
    start = datetime.now()
    print(f'{start.strftime("%Y-%m-%d %H:%M:%S")}')
    return start

def endtimer(start):
    end = datetime.now()
    print(f'{end.strftime("%Y-%m-%d %H:%M:%S")}')
    durn = end - start
    print(f'Duration: {durn.total_seconds()}s')
    return end, durn

In [392]:
labels =  np.array([
                    [ # Patch 1
                            [1,2,3],
                            [11,12,13],
                            [21,22,23]
                    ],
                    [ # Patch 2
                            [4,5,6],
                            [14,15,16],
                            [24,25,26]
                    ],
                    [ # Patch 3
                            [7,8,9],
                            [17,18,19],
                            [27,28,29]
                    ],
                    [ # Patch 4
                            [31,32,33],
                            [41,42,43],
                            [51,52,53]
                    ],
                    [ # Patch 5
                            [34,35,36],
                            [44,45,46],
                            [54,55,56]
                    ],
                    [ # Patch 6
                            [37,38,39],
                            [47,48,49],
                            [57,58,59]
                    ],
                    [ # Patch 7
                            [61,62,63],
                            [71,72,73],
                            [81,82,83]
                    ],
                    [ # Patch 8
                            [64,65,66],
                            [74,75,76],
                            [84,85,86]
                    ],
                    [ # Patch 9
                            [67,68,69],
                            [77,78,79],
                            [87,88,89]
                    ],
                ])

In [394]:
# Combine multiple sets of tiles patches to form batch input for testing
patch_dim = labels.shape[-1]  # test [3,3] | project [61,61]
tile_dim = 9                        # test 9 or 6| project [388,388]
number_of_tiles_in_batch = 12       # test 12 or 27    | calculated from batch

batch_labels = np.vstack((labels,labels,labels,labels,labels,labels, labels,labels,labels,labels,labels,labels)).reshape(108,patch_dim, patch_dim)

print(labels.shape)
print(labels.size)
print(batch_labels.shape)
print(batch_labels.size)

(9, 3, 3)
81
(108, 3, 3)
972


## Aggregate Patches into Tiles

In [395]:
num_tiles =  batch_labels.size // (tile_dim*tile_dim)

tiles =  np.empty(shape=(0,tile_dim,tile_dim))
for t in range(num_tiles):
    f_dim = (tile_dim // patch_dim)
    f_dim_sq = f_dim*f_dim
    batch_tile = batch_labels[(f_dim_sq*t):(f_dim_sq*(t+1))]
    #print(batch_tile)

    batch_re = batch_tile.swapaxes(0,1) # orig but doesn't work for 6
    #print(batch_re)

    batch_re = batch_re.reshape(tile_dim,tile_dim)
    #print(batch_re)

    tile = np.empty(shape=(0,tile_dim))
    for i in range(0,f_dim):
        for j in range(0,patch_dim):
            tile = np.vstack(( tile, batch_re[i+f_dim*j] ))

    tiles = np.vstack(( tiles, [tile] ))

#print(f'Tile # {t+1}')
print(tiles)
print(tiles.shape)
#print('-'*80)

[[[ 1.  2.  3.  4.  5.  6.  7.  8.  9.]
  [11. 12. 13. 14. 15. 16. 17. 18. 19.]
  [21. 22. 23. 24. 25. 26. 27. 28. 29.]
  [31. 32. 33. 34. 35. 36. 37. 38. 39.]
  [41. 42. 43. 44. 45. 46. 47. 48. 49.]
  [51. 52. 53. 54. 55. 56. 57. 58. 59.]
  [61. 62. 63. 64. 65. 66. 67. 68. 69.]
  [71. 72. 73. 74. 75. 76. 77. 78. 79.]
  [81. 82. 83. 84. 85. 86. 87. 88. 89.]]

 [[ 1.  2.  3.  4.  5.  6.  7.  8.  9.]
  [11. 12. 13. 14. 15. 16. 17. 18. 19.]
  [21. 22. 23. 24. 25. 26. 27. 28. 29.]
  [31. 32. 33. 34. 35. 36. 37. 38. 39.]
  [41. 42. 43. 44. 45. 46. 47. 48. 49.]
  [51. 52. 53. 54. 55. 56. 57. 58. 59.]
  [61. 62. 63. 64. 65. 66. 67. 68. 69.]
  [71. 72. 73. 74. 75. 76. 77. 78. 79.]
  [81. 82. 83. 84. 85. 86. 87. 88. 89.]]

 [[ 1.  2.  3.  4.  5.  6.  7.  8.  9.]
  [11. 12. 13. 14. 15. 16. 17. 18. 19.]
  [21. 22. 23. 24. 25. 26. 27. 28. 29.]
  [31. 32. 33. 34. 35. 36. 37. 38. 39.]
  [41. 42. 43. 44. 45. 46. 47. 48. 49.]
  [51. 52. 53. 54. 55. 56. 57. 58. 59.]
  [61. 62. 63. 64. 65. 66. 67. 68. 6

In [396]:
# convert RGB components of (B, T, C, H, W) to grayscale (B, H, W)
batch = np.empty(shape=(3,6,4,61,61))
